**GoogleNetModel **

In [ ]:
!pip install spectral
!pip install vit-keras
!pip install tensorflow-addons
# Loading Drive for Colab
from google.colab import drive
import urllib
from google.colab import files
import os
import time
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io as sio
import seaborn as sns
import spectral
import spectral.io.envi as envi
from sklearn.decomposition import (IncrementalPCA, KernelPCA, PCA, SparsePCA,
                                   TruncatedSVD)
from sklearn.metrics import (accuracy_score, classification_report,
                             cohen_kappa_score, confusion_matrix)
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.layers import (Conv3D,Conv2D, Dense, Dropout, Flatten, Input,
                          Reshape,MaxPooling2D)
from keras.losses import categorical_crossentropy
from keras.models import Model, Sequential
# from keras.utils import np_utils
from tensorflow.keras.layers import (Activation, Lambda, multiply)
from tensorflow.keras.optimizers import Adam
from keras.optimizers import Adam
from tensorflow.keras.optimizers import legacy
from operator import truediv
from vit_keras import utils, vit
# from keras.utils.vis_utils import plot_model
from keras.utils import to_categorical

In [ ]:
# Mounting the Colab Drive for Loading Datasets
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Loading Hyperspectral Datasets
def LoadHSIData(method):
    HSI_url = ['http://www.ehu.eus/ccwintco/uploads/6/67/Indian_pines_corrected.mat',
               'http://www.ehu.eus/ccwintco/uploads/c/c4/Indian_pines_gt.mat']
    for i in range(2):
        file_name = HSI_url[i].split('/')[-1]
        data_path = os.path.join(os.getcwd(),'/content/drive/My Drive/'+str(file_name))
        if os.path.exists(data_path) == False:
            print("Downloading data file from %s to %s" % (HSI_url[i], data_path))
            urllib.request.urlretrieve(url=HSI_url[i], filename=data_path)
            print(str(file_name)+" is Successfully downloaded")
        else:
            print(str(file_name) + " already exists")
    print('All HSI dataset exist!')
    ## Loading Datasets
    data_path = os.path.join(os.getcwd(),'/content/drive/My Drive/')
    if method == 'IP':
        HSI = sio.loadmat(os.path.join(data_path, 'Indian_pines_corrected.mat'))['indian_pines_corrected']
        GT = sio.loadmat(os.path.join(data_path, 'Indian_pines_gt.mat'))['indian_pines_gt']
        Num_Classes = 16
    return HSI, GT, Num_Classes

In [ ]:
## Different Dimensional Reduction Methods
def DLMethod(method, HSI, NC = 75):
    RHSI = np.reshape(HSI, (-1, HSI.shape[2]))
    if method == 'PCA': ## PCA
        pca = PCA(n_components = NC, whiten = True)
        RHSI = pca.fit_transform(RHSI)
        RHSI = np.reshape(RHSI, (HSI.shape[0], HSI.shape[1], NC))
    return RHSI

In [ ]:
def TrTeSplit(HSI, GT, trRatio, vrRatio, teRatio, randomState=345):
    # Split into train and test sets
    Tr, Te, TrC, TeC = train_test_split(HSI, GT, test_size=teRatio,
                                        random_state=randomState, stratify=GT)
    # Calculate the validation ratio based on the updated test and train ratios
    totalTrRatio = trRatio + vrRatio
    new_vrRatio = vrRatio / totalTrRatio
    # Split train set into train and validation sets
    Tr, Va, TrC, VaC = train_test_split(Tr, TrC, test_size=new_vrRatio,
                                        random_state=randomState, stratify=TrC)

    return Tr, Va, Te, TrC, VaC, TeC

In [ ]:
## Global Parameters for Loop
HSID = "IP"
DLM = "PCA"
WS = 9      ## 3, 5, 7, 9, 11, 13, 15
teRatio = 0.20
vrRatio = 0.50
trRatio = 0.50
k = 15
adam = tf.keras.optimizers.legacy.Adam(lr = 0.0001, decay = 1e-06)
epochs = 50
batch_size = 56

In [ ]:
## 3-D HSI slices
def ImageCubes(HSI, GT, WS = WS, removeZeroLabels = True):
    margin = int((WS - 1) / 2)
    zeroPaddedX = ZeroPad(HSI, margin = margin)
    ## split patches
    patchesData = np.zeros((HSI.shape[0] * HSI.shape[1], WS, WS, HSI.shape[2]))
    patchesLabels = np.zeros((HSI.shape[0] * HSI.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = GT[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels
## Padding around HSI
def ZeroPad(HSI, margin = 2):
    NHSI = np.zeros((HSI.shape[0] + 2 * margin, HSI.shape[1] + 2* margin, HSI.shape[2]))
    x_offset = margin
    y_offset = margin
    NHSI[x_offset:HSI.shape[0] + x_offset, y_offset:HSI.shape[1] + y_offset, :] = HSI
    return NHSI
## Compute the Patch to Prepare for Ground Truths
def Patch(HSI,height_index,width_index):
    height_slice = slice(height_index, height_index+WS)
    width_slice = slice(width_index, width_index+WS)
    patch = HSI[height_slice, width_slice, :]
    return patch

In [ ]:
## Assigning Class Labels for Final Classification and Confusion Matrices
def ClassificationReports(TeC, HSID, Te_Pred):
    Te_Pred = np.argmax(Te_Pred, axis=1)
    if HSID == 'IP':
        target_names = ['Alfalfa', 'Corn-notill', 'Corn-mintill', 'Corn'
                        ,'Grass-pasture', 'Grass-trees', 'Grass-mowed',
                        'Hay-windrowed', 'Oats', 'Soybean-notill', 'Soybean-mintill',
                        'Soybean-clean', 'Wheat', 'Woods', 'Buildings',
                        'Stone-Steel']
    elif HSID == 'SA':
        target_names = ['Weeds_1','Weeds_2','Fallow',
                        'Fallow_rough_plow','Fallow_smooth', 'Stubble','Celery',
                        'Grapes_untrained','Soil_vinyard_develop','Corn_Weeds',
                        'Lettuce_4wk','Lettuce_5wk','Lettuce_6wk',
                        'Lettuce_7wk', 'Vinyard_untrained','Vinyard_trellis']
    elif HSID == 'PU':
        target_names = ['Asphalt','Meadows','Gravel','Trees', 'Painted','Soil','Bitumen',
                        'Bricks','Shadows']
    elif HSID == 'KSC':
        target_names = ['Scrub', 'Willow Swamp', 'CP/Oak', 'CP hammock', 'Slash Pine', 'Oak/Broadleaf', 'Hardwood Swamp',
                        'Graminoid Marsh', 'Spartina Marsh', 'Cattail Marsh', 'Salt Marsh', 'Mud Flats', 'Water']
    elif HSID == 'BS':
        target_names = ['Water', 'Hippo Grass', 'Floodplain Grasses 1', 'Floodplain Grasses 2',
                        'Reeds 1', 'Riparian', 'Firescar 2', 'Island Interior', 'Woodlands',
                        'Acacia Shrublands', 'Acacia Grasslands', 'Short Mopane', 'Mixed Mopane', 'Exposed Soils']
    elif HSID == 'PC':
        target_names = ['Water', 'Trees', 'Asphalt', 'Bricks', 'Bitumen', 'Tiles', 'Shadows',
                        'Meadows', 'Soil']
    elif HSID == 'SLA':
        target_names = ['Brocoli 1', 'Corn weeds', 'Lettuce 4wk', 'Lettuce 5wk',
                       'Lettuce 6wk', 'Lettuce 7wk']
    elif HSID == 'PC':
        target_names = ['Water', 'Trees', 'Asphalt', 'Bricks', 'Bitumen', 'Tiles', 'Shadows',
                        'Meadows', 'Soil']
    elif HSID == 'UH':
        target_names = ['Healthy grass', 'Stressed grass', 'Synthetic grass', 'Trees',
                    'Soil', 'Water', 'Residential', 'Commercial', 'Road',
                    'Highway', 'Railway', 'Parking Lot 1', 'Parking Lot 2',
                    'Tennis Court', 'Running Track']
    classification = classification_report(np.argmax(TeC, axis=1), Te_Pred, target_names = target_names)
    oa = accuracy_score(np.argmax(TeC, axis=1), Te_Pred)
    confusion = confusion_matrix(np.argmax(TeC, axis=1), Te_Pred)
    list_diag = np.diag(confusion)
    list_raw_sum = np.sum(confusion, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    aa = np.mean(each_acc)
    kappa = cohen_kappa_score(np.argmax(TeC, axis=1), Te_Pred)
    return classification, confusion, oa*100, each_acc*100, aa*100, kappa*100, target_names

## Writing Results in CSV files
def CSVResults(file_name, classification, confusion, Tr_Time, Te_Time, DL_Time, kappa, oa, aa, each_acc):
    classification = str(classification)
    confusion = str(confusion)
    with open(file_name, 'w') as CSV_file:
      CSV_file.write('{} Tr_Time'.format(Tr_Time))
      CSV_file.write('\n')
      CSV_file.write('{} Te_Time'.format(Te_Time))
      CSV_file.write('\n')
      CSV_file.write('{} DL_Time'.format(DL_Time))
      CSV_file.write('\n')
      CSV_file.write('{} Kappa accuracy (%)'.format(kappa))
      CSV_file.write('\n')
      CSV_file.write('{} Overall accuracy (%)'.format(oa))
      CSV_file.write('\n')
      CSV_file.write('{} Average accuracy (%)'.format(aa))
      CSV_file.write('\n')
      CSV_file.write('{}'.format(classification))
      CSV_file.write('\n')
      CSV_file.write('{}'.format(each_acc))
      CSV_file.write('\n')
      CSV_file.write('{}'.format(confusion))
    return CSV_file

## Plot and Save Confusion Matrix
def Conf_Mat(Te_Pred, TeC, target_names):
    plt.rcParams.update({'font.size': 12})
    Te_Pred = np.argmax(Te_Pred, axis=1)
    confusion = confusion_matrix(np.argmax(TeC, axis=1), Te_Pred, labels=np.unique(np.argmax(TeC, axis=1)))
    cm_sum = np.sum(confusion, axis=1, keepdims=True)
    cm_perc = confusion / cm_sum.astype(float) * 100
    annot = np.empty_like(confusion).astype(str)
    nrows, ncols = confusion.shape
    for l in range(nrows):
      for m in range(ncols):
        c = confusion[l, m]
        p = cm_perc[l, m]
        if l == m:
          s = cm_sum[l]
          annot[l, m] = '%.1f%%\n%d/%d' % (p, c, s)
        elif c == 0:
          annot[l, m] = ''
        else:
          annot[l, m] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(confusion, index=np.unique(target_names), columns=np.unique(target_names))
    return cm, annot

## Plot Ground Truths
def GT_Plot(RDHSI, GT, model, WS, k):
  RDHSI = RDHSI.reshape(-1, WS, WS, k, 1)
  Predicted = model.predict(RDHSI)
  Predicted = np.argmax(Predicted, axis=1)
  height, width = np.shape(GT)
  ## Calculate the predicted Ground Truths
  outputs = np.zeros((height, width))
  count = 0
  for AA in range(height):
    for BB in range(width):
      target = int(GT[AA,BB])
      if target == 0:
        continue
      else:
        outputs[AA][BB] = Predicted[count]
        count = count+1
  return outputs

In [ ]:
## Main Function to load Datasets, Dimensional Reduction and Creating Patchs for CNN
HSI, GT, Num_Classes = LoadHSIData(HSID)
## Reduce the Dimensionality
#if k < HSI.shape[2]:
start = time.time()
RDHSI = DLMethod(DLM, HSI, NC = k)
end = time.time()
DL_Time = end - start
## Create Image Cubes for Model Building
CRDHSI, CGT = ImageCubes(RDHSI, GT, WS = WS)
## Split Train/validation and Test sets
Tr, Va, Te, TrC, VaC, TeC = TrTeSplit(CRDHSI, CGT, trRatio, vrRatio, teRatio)
# Reshape train, validation, and test sets
Tr = Tr.reshape(-1, WS, WS, k, 1)
TrC = to_categorical(TrC)
Va = Va.reshape(-1, WS, WS, k, 1)
VaC = to_categorical(VaC)
Te = Te.reshape(-1, WS, WS, k, 1)
TeC = to_categorical(TeC)

Indian_pines_corrected.mat already exists
Indian_pines_gt.mat already exists
All HSI dataset exist!


In [ ]:
def GoogleNetModel(WS, k, Num_Classes):
    input_shape = (WS, WS, k)
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(Num_Classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [ ]:
def GoogleNetModel(input_shape, num_classes):
    # Define your custom GoogleNet-like model architecture here
    input_tensor = Input(shape=input_shape)

    # Example: stack convolutional and pooling layers
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_tensor)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    # Add more layers as per GoogleNet architecture

    # Flatten and add Dense layers for classification
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=output)
    return model

In [ ]:
def train_and_evaluate_model(model_name, Tr, TrC, Va, VaC, Te, TeC, adam, CRDHSI, HSID, teRatio, k, WS, DLM, RDHSI, GT,Num_Classes,batch_size,epochs):

    # Calling Custom Model
    model = model_name(WS, k, Num_Classes)
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

    # Training the model
    start = time.time()
    history = model.fit(x=Tr, y=TrC, batch_size=batch_size, epochs=epochs, validation_data=(Va, VaC))
    end = time.time()
    Tr_Time = end - start

    # Predicting with the model
    start = time.time()
    Te_Pred = model.predict(Te)
    end = time.time()
    Te_Time = end - start

    # Classification Report
    classification,confusion,oa,each_acc,aa,kappa,target_names = ClassificationReports(TeC, HSID, Te_Pred)
    print(classification)
    # file_name = f"str(HSID)+str(teRatio)+str(WS)+str(DLM)+str(k)+Report_{model_name.__name__}.csv"
    file_name = f"{HSID}_{teRatio}_{k}_{WS}_{DLM}_Report_{model_name.__name__}.csv"
    CSV_file = CSVResults(file_name, classification, confusion, Tr_Time, Te_Time, DL_Time, kappa, oa, aa, each_acc)
    files.download(file_name)

    # Confusion Matrix
    cm, annot = Conf_Mat(Te_Pred, TeC, target_names)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=(15,15))
    sns.heatmap(cm, cmap= "Spectral", annot=annot, fmt='', ax=ax, linewidths=0.5)
    file_name = f"{HSID}_{teRatio}_{k}_{WS}_{DLM}_Confusion_Matrix_{model_name.__name__}.png"
    plt.savefig(file_name, dpi=500)
    files.download(file_name)

    # Ground Truths
    outputs = GT_Plot(CRDHSI, GT, model, WS, k)
    plt.figure(figsize=(10, 10))
    plt.imshow(outputs, cmap='nipy_spectral')
    plt.axis('off')
    file_name = f"{HSID}_{teRatio}_{k}_{WS}_{DLM}_Ground_Truths_{model_name.__name__}.png"
    plt.savefig(file_name, dpi=500)
    files.download(file_name)
    return history

In [ ]:
def train_and_evaluate_model(model_name, Tr, TrC, Va, VaC, Te, TeC, adam, CRDHSI, HSID, teRatio, k, WS, DLM, RDHSI, GT, Num_Classes, batch_size, epochs):
    # Calling Custom Model with input_shape and num_classes
    model = model_name((WS, WS, k), Num_Classes)
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])


In [ ]:
model_names = [GoogleNetModel]  # Add your model names here

# Loop over the model names and run train_and_evaluate_model() for each model
history_list = []
for model_name in model_names:
    history = train_and_evaluate_model(
        model_name, Tr, TrC, Va, VaC, Te, TeC,
        adam, CRDHSI, HSID, teRatio, k, WS, DLM, RDHSI, GT,
        Num_Classes, batch_size, epochs  # Add epochs here
    )
    history_list.append(history)



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9, 9, 15)]        0         
                                                                 
 conv2d (Conv2D)             (None, 9, 9, 64)          8704      
                                                                 
 max_pooling2d (MaxPooling2  (None, 4, 4, 64)          0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 1024)              1049600   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                             